https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/tutorials/mnist/beginners/

MNIST 데이터셋은 Yann LeCun의 웹사이트에 호스팅되어 있습니다. 좀 편하게 하기 위해서, 우리는 이 데이터를 다운로드받고 설치하는 파이썬 코드를 넣어놨습니다. 여기에서 코드를 다운받은 후 아래와 같이 코드를 불러올 수도 있습니다. 아니면 그냥 복사하고 붙여넣기를 하십시오.

In [11]:
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

다운로드된 데이터는 55,000개의 학습 데이터(mnist.train), 10,000개의 테스트 데이터(mnist.test), 그리고 5,000개의 검증 데이터(mnist.validation) 이렇게 세 부분으로 나뉩니다. 데이터가 이렇게 나뉜다는 것은 매우 중요합니다. 왜냐하면 우리가 학습시키지 않는 데이터를 통해, 우리가 학습한 것이 정말로 일반화되었다고 확신할 수 있기 때문입니다!

앞서 언급했듯이, 각 MNIST 데이터셋은 두 부분으로 나뉩니다. 손으로 쓴 숫자와 그에 따른 라벨입니다. 우리는 이미지를 "xs"라고 부르고 라벨을 "ys"라고 부를 것입니다. 학습 데이터셋과 테스트 데이터셋은 둘 다 xs와 ys를 가집니다. 예를 들어, 학습 이미지는 mnist.train.images이며, 학습 라벨은 mnist.train.labels입니다.

각 이미지는 28x28 픽셀입니다. 우리는 이를 숫자의 큰 배열로 해석할 수 있습니다.

우리는 이 배열을 펼쳐서 28x28 = 784 개의 벡터로 만들 수 있습니다. 이미지들 간에 일관적으로 처리하기만 한다면, 배열을 어떻게 펼치든지 상관없습니다. 이러한 관점에서, MNIST 이미지는 매우 호화스러운 구조(주의 : 연산을 많이 요하는 시각화입니다)를 가진, 단지 784차원 벡터 공간에 있는 여러 개의 데이터일 뿐입니다.

데이터를 펼친 결과로 mnist.train.images는 [55000, 784]의 형태를 가진 텐서(n차원 배열)가 됩니다. 첫 번째 차원은 이미지를 가리키며, 두 번째 차원은 각 이미지의 픽셀을 가르킵니다. 텐서의 모든 성분은 특정 이미지의 특정 픽셀을 특정하는 0과 1사이의 픽셀 강도입니다.

MNIST에서 각각에 대응하는 라벨은 0과 9사이의 숫자이며, 각 이미지가 어떤 숫자인지를 말해줍니다. 이 튜토리얼의 목적을 위해서 우리는 라벨을 "원-핫 벡터"로 바꾸길 원합니다. 원-핫 벡터는 단 하나의 차원에서만 1이고, 나머지 차원에서는 0인 벡터입니다. 이 경우, n번째 숫자는 n번째 차원이 1인 벡터로 표현될 것입니다. 예를 들어서, 3은 [0,0,0,1,0,0,0,0,0,0]입니다. 결과적으로, mnist.train.labels는 [55000, 10]의 모양을 같은 실수 배열이 됩니다.(역자 주 : 정수 배열이 아니라, 실수 배열로 취급하는 데에는 이후 소프트맥스 회귀의 결과가 정수형이 아닌 실수형으로 산출되기 때문입니다.)

텐서플로우를 사용하기 위해서는 이를 임포트해야 합니다.

In [1]:
import tensorflow as tf

/home/zog/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


우리는 이 상호작용하는 연산들을 심볼릭 변수를 활용해 기술하게 됩니다. 하나 만들어 보죠:

In [2]:
x = tf.placeholder(tf.float32, [None, 784])

x에 특정한 값이 주어진 것은 아닙니다. 이는 'placeholder'로, 우리가 텐서플로우에서 연산을 실행할 때 값을 입력할 자리입니다. 여기서는 784차원의 벡터로 변형된 MNIST 이미지의 데이터를 넣으려고 합니다. 우린 이걸 [None, 784]의 형태를 갖고 부동소수점으로 이루어진 2차원 텐서로 표현합니다. (여기서 None은 해당 차원의 길이가 어떤 길이든지 될 수 있음을 의미합니다)

또한 우리의 모델에는 가중치와 바이어스 역시 필요합니다. 우리는 이를 부가적인 입력처럼 다루는 방법을 생각할 수도 있지만, 텐서플로우는 Variable이라고 불리는 보다 나은 방법을 갖고 있습니다. Variable은 서로 상호작용하는 연산으로 이루어진 텐서플로우 그래프 안에 존재하는, 수정 가능한 텐서입니다. Variable은 연산에 사용되기도 하고, 연산을 통해 수정되기도 합니다. 머신러닝에 이를 사용할 때에는 주로 모델의 변수를 Variable들로 사용하게 됩니다.

In [3]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

우리는 tf.Variable에 Variable의 초기값을 넘겨줌으로써 이 Variable들을 생성합니다: 여기서는 W와 b 둘 다 0으로 이루어진 텐서로 초기화를 합니다. 이제부터 W와 b를 학습해 나갈 것이므로, 각각의 초기값은 크게 중요하지 않습니다.

W가 [784, 10]의 형태를 갖는 것에 주목해주시기 바랍니다. 이러한 형태로 만든 이유는 W에 784차원의 이미지 벡터를 곱해서 각 클래스에 대한 증거값을 나타내는 10차원 벡터를 얻고자 하기 때문입니다. b는 그 10차원 벡터에 더하기 위해 [10]의 형태를 갖는 것입니다.

이제 우리는 모델을 구현할 수 있습니다. 단 한줄로요!

우선, tf.matmul(x, W)로 x와 W를 곱합니다. 이 표현은 위에서 본 수식에서 곱했던 순서인 Wx와 반대인데 (행렬이므로 순서가 중요하죠), x가 여러 입력값을 갖는 2차원 텐서인 경우에도 대응하기 위한 작은 트릭입니다. 그 다음엔 b를 더하고, 마지막으로 tf.nn.softmax을 적용합니다.

In [4]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

우리의 모델을 학습시키기 위해서는 우선 모델이 좋다는 것은 어떤 것인지를 정의해야 합니다. 사실 머신러닝에서는 모델이 안좋다는 것이 어떤 의미인지를 주로 정의합니다. 우리는 이를 주로 비용(cost) 또는 손실(loss)이라고 부르며, 이것들은 우리의 모델이 우리가 원하는 결과에서 얼마나 떨어져있는지를 보여주는 값입니다. 우리는 그 격차를 줄이기 위해 노력하며, 그 격차가 적으면 적을수록 우리의 모델은 좋다고 말합니다.

모델의 손실을 정의하기 위해 자주 사용되는 좋은 함수 중 하나로 "크로스 엔트로피"가 있습니다. 원래 크로스 엔트로피는 정보 이론 분야에서 정보를 압축하는 방법으로써 고안된 것이지만, 현재는 도박에서 머신러닝에 이르기까지 여러 분야에서 중요한 아이디어로 사용되고 있습니다. 크로스 엔트로피는 다음과 같이 정의됩니다:

y는 우리가 예측한 확률 분포이며, y'는 실제 분포(우리가 입력하는 원-핫 벡터) 입니다. 대략적으로 설명하자면, 크로스 엔트로피는 우리의 예측이 실제 값을 설명하기에 얼마나 비효율적인지를 측정하는 것입니다. 크로스 엔트로피에 대해서 더 자세하게 다루는 것은 이 튜토리얼의 범위를 벗어나는 내용입니다만, 알아둘 가치는 있습니다.

크로스 엔트로피를 구현하기 위해서는 올바른 답을 넣기 위한 새로운 placeholder를 추가하는 것 부터 시작해야 합니다.

In [5]:
y_ = tf.placeholder(tf.float32, [None, 10])

이제 우리는 크로스 엔트로피를 구현할 수 있습니다:

In [6]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

우선, tf.log는 y의 각 원소의 로그 값을 계산합니다. 그 다음, y_의 각 원소를 tf.log(y)의 해당하는 원소들과 곱합니다. 그리고 tf.reduce_sum으로 y의 2번째 차원(reduction_indices=[1]이라는 파라미터가 주어졌으므로)의 원소들을 합합니다. 마지막으로, tf.reduce_mean으로 배치(batch)의 모든 예시에 대한 평균을 계산합니다.

(수학적으로 불안정한 계산이기 때문에, 소스 코드에서는 이 연산을 사용하지 않고 있는 것에 주의하시기 바랍니다. 대신, 정규화 되지 않은 로짓(logit)에 대해 tf.nn.softmax_cross_entropy_with_logits을 적용합니다(즉, tf.matmul(x, W) + b)에 softmax_cross_entropy_with_logits을 사용합니다). 이렇게 하는 이유는 이 수학적으로 보다 안정적인 함수가 내부적으로 소프트맥스 활성을 계산하기 때문입니다. 당신의 코드에서도 tf.nn.(sparse_)softmax_cross_entropy_with_logits를 사용하는 것을 고려해보시기 바랍니다.)

우리의 모델이 할 일을 우리가 알고있다면, 이를 텐서플로우를 통해 학습시키는 것은 매우 간단합니다. 텐서플로우는 당신이 하고자 하는 연산의 전체 그래프를 알고 있으므로, 손실(당신이 최소화 하고 싶어하는 것이죠)에 당신이 설정한 변수들이 어떻게 영향을 주는지를 역전파(backpropagation) 알고리즘을 자동으로 사용하여 매우 효율적으로 정의할 수 있기 때문입니다. 그리고나서 텐서플로우는 당신이 선택한 최적화 알고리즘을 적용하여 변수를 수정하고 손실을 줄일 수 있습니다.

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

여기서는 텐서플로우에게 학습 비율 0.5로 경사 하강법(gradient descent algorithm)을 적용하여 크로스 엔트로피를 최소화하도록 지시합니다. 경사하강법이란 텐서플로우가 각각의 변수를 비용을 줄이는 방향으로 조금씩 이동시키는 매우 단순한 방법입니다. 그러나 텐서플로우는 다른 여러 최적화 알고리즘을 제공합니다: 그 중 하나를 적용하는 것은 코드 한 줄만 수정하면 될 정도로 간단합니다.

여기서 텐서플로우가 실제로 뒤에서 하는 일은, 역전파와 경사하강이라는 새로운 작업을 당신의 그래프에 추가하는 것입니다. 이제 텐서플로우가 실행되면 비용을 감소시키기 위해 변수들을 살짝 수정하는 경사 하강 학습 작업 한 번을 돌려줄 것입니다.

이제 우리 모델은 학습할 준비가 되었습니다. 학습을 실행시키기 전에 마지막으로, 우리가 작성한 변수들을 초기화하는 작업을 추가해야 합니다:

In [8]:
init = tf.global_variables_initializer()

이제 Session에서 모델을 실행시키고, 변수들을 초기화 하는 작업을 실행시킬 수 있습니다:

In [9]:
sess = tf.Session()
sess.run(init)

학습을 시킵시다 -- 여기선 학습을 1000번 시킬 겁니다!

In [12]:
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

반복되는 루프의 각 단계마다, 우리는 학습 데이터셋에서 무작위로 선택된 100개의 데이터로 구성된 "배치(batch)"를 가져옵니다. 그 다음엔 placeholder의 자리에 데이터를 넣을 수 있도록 train_step을 실행하여 배치 데이터를 넘깁니다.

무작위 데이터의 작은 배치를 사용하는 방법을 확률적 학습(stochastic training)이라고 부릅니다 -- 여기서는 확률적 경사 하강법입니다. 이상적으로는 학습의 매 단계마다 전체 데이터를 사용하고 싶지만(그렇게 하는게 우리가 지금 어떻게 하는게 좋을지에 대해 더 잘 알려줄 것이므로), 그렇게 하면 작업이 무거워집니다. 따라서 그 대신에 매번 서로 다른 부분집합을 사용하는 것입니다. 이렇게 하면 작업 내용은 가벼워지지만 전체 데이터를 쓸 때의 이점은 거의 다 얻을 수 있기 때문입니다.

우리가 작성한 모델은 성능이 어느 정도일까요?

흐음, 우선 모델이 라벨을 올바르게 예측했는지 확인해봅시다. tf.argmax는 텐서 안에서 특정 축을 따라 가장 큰 값의 인덱스를 찾기에 매우 유용한 함수입니다. 예를 들면, tf.argmax(y,1)는 우리의 모델이 생각하기에 각 데이터에 가장 적합하다고 판단한(가장 증거값이 큰) 라벨이며, tf.argmax(y_,1)는 실제 라벨입니다. 우리는 tf.equal을 사용하여 우리의 예측이 맞았는지 확인할 수 있습니다.

In [13]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

이렇게 하면 부울 값으로 이루어진 리스트를 얻게 됩니다. 얼마나 많이 맞았는지 판단하려면, 이 값을 부동소수점 값으로 변환한 후 평균을 계산하면 됩니다. 예를 들면, [True, False, True, True]는 [1,0,1,1]로 환산할 수 있고, 이 값의 평균을 계산하면 0.75가 됩니다.

In [14]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

마지막으로, 우리의 테스트 데이터를 대상으로 정확도를 계산해 봅시다.

In [15]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.914


결과는 약 92% 정도가 나올 것입니다.

좋은 결과일까요? 글쎄요, 딱히 그렇진 않습니다. 사실, 매우 안좋은 결과입니다. 왜냐하면 우리가 매우 단순한 모델을 사용했기 때문입니다. 약간만 바꾸면, 97%의 정확도를 얻을 수 있습니다. 가장 좋은 모델은 정확도가 99.7%도 넘을 수 있지요! (더 알고 싶으시다면 다음의 결과 목록을 확인해보세요)

여기서 중요한 것은 우리가 이 모델을 통해 배운 것입니다. 혹시 아직도 이 결과가 조금 실망스러우시면 다음 튜토리얼을 읽어보시기 바랍니다. 거기선 우리가 훨씬 더 좋은 결과값도 얻고, 텐서플로우로 더 복잡한 모델을 작성하는 방법도 배우게 된답니다!